In [205]:
import requests
import random
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime , timedelta
import re
import os



path = "//Users//hajiaga//Desktop//DataScience//chromedriver"

driver = webdriver.Chrome()

driver.implicitly_wait(5)


In [206]:
def azetoengdate(string):

    gun = string[:2]

    ay = str(re.findall(r'\D+', string)).replace(" ","").lower()

    if ay == "yanvar":
        numay = "01"
    elif ay == "fevral":
        numay = "02"
    elif ay == "mart":
        numay = "03"
    elif ay == "aprel":
        numay = "04"
    elif ay == "may":
        numay = "05"
    elif ay == "iyun":
        numay = "06"
    elif ay == "iyul":
        numay = "07"
    elif ay == "avqust":
        numay = "08"
    elif ay == "sentyabr":
        numay = "09"
    elif ay == "oktyabr":
        numay = "10"
    elif ay == "noyabr":
        numay = "11"
    elif ay == "dekabr":
        numay = "12"
    else:
        numay ="Nan"
    
    return f"{numay} {gun}"

In [207]:
def scrape_new_window(href , dic):
    time.sleep(1)
    driver.get(href)

    dic["full_name"].append((driver.find_element(By.XPATH , "//h1[@class = 'product-title']")).text)

    temp = driver.find_element(By.XPATH , "//div[@class = 'product-properties__column']")

    dic["location"].append((temp.find_element(By.XPATH , "//span[@class = 'product-properties__i-value']")).text)

    dic["comp_name"].append((temp.find_element(By.TAG_NAME , "a")).text)

    tempp = driver.find_element(By.XPATH , "//div[@class = 'product-price__i product-price__i--bold']")
    
    temppspans = tempp.find_elements(By.TAG_NAME , "span")

    dic["price"].append(temppspans[0].text)

    dic["currency"].append(temppspans[1].text)

    temp1 = driver.find_element(By.XPATH , "//div[@class = 'product-info product-info__statistics']")

    temp2 = temp1.find_elements(By.TAG_NAME , "div")

    date = (str(temp2[1].find_element(By.TAG_NAME , "span").text)).split(',')[0]

    if date == "Bugün":
        today = datetime.today()

        dic["date"].append(today.strftime('%m %d'))
        
    elif date == "Dünən":

        yesterday = datetime.today() - timedelta(days=1)

        dic["date"].append(yesterday.strftime('%B %d'))
    else:
        dic["date"].append(azetoengdate(date))

    driver.back()

    return dic


In [208]:
path1 = "https:/tap.az"

driver.get(path1)

time.sleep(5)
try:
    category = driver.find_element(By.XPATH , "//a[@data-for = 'consumer-electronics']")

    driver.get(category.get_attribute('href'))

    # button = driver.find_element(By.XPATH , "//button[@class = 'filter-dropdown' and @type = 'submit']")

    # button.click()

    dropdown = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'price']")

    dropdown.click()
    
    minprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_lower']")

    maxprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_upper']")

    minprice.send_keys("1350")

    maxprice.send_keys("2500")

    time.sleep(5)

    categories = driver.find_element(By.XPATH , "//div[@class = 'subcategories-inner']")

    uls = categories.find_elements(By.TAG_NAME , 'ul')

    for ul in uls:
        
        lis = ul.find_elements(By.TAG_NAME , 'li')

        for li in lis:

            if "noutbuk" in str(li.find_element(By.TAG_NAME , 'a').get_attribute('href')).lower():

                href = li.find_element(By.TAG_NAME , 'a').get_attribute('href')

                break
    
    driver.get(href)

    dropdown_new = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'new']")

    dropdown_new.click()

    dropdown_new.find_element(By.TAG_NAME,"ul").find_elements(By.TAG_NAME , 'li')[1].click()
    
    time.sleep(10)

    product_columns = driver.find_element(By.XPATH ,"//div[@class = 'js-endless-container products endless-products']" )

    time.sleep(3)

    products = product_columns.find_elements(By.XPATH, "./div")

    dic_comp = {}

    dic_comp["location"] = []

    dic_comp["date"] = []

    dic_comp["link"] = []

    dic_comp["price"] = []

    dic_comp["comp_name"] = []

    dic_comp["full_name"] = []

    dic_comp["currency"] = []

    i = 1
    x = 0
    for product in products:
        i +=1
        if i >47:
            time.sleep(3)
        
            a = product.find_element(By.XPATH , "//a[@class = 'products-link']")

            dic_comp["link"].append(a.get_attribute("href"))

            dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)
    # The final problem is that i count until 48 but after that website is getting refreshed so i need to make a function for that and stop it gettinf the same thing after 48
            



    
        
finally:
    driver.close()



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


KeyboardInterrupt: 

0


In [ ]:
dic_comp["link"][2]

'https://tap.az/elanlar/elektronika/noutbuklar?ajax=1&cursor=1711522266.540859_41020022&order=&p%5B769%5D=false&p%5B822%5D=&p%5B858%5D=&q%5Bcontact_id%5D=&q%5Bkeywords%5D=&q%5Bprice%5D%5B%5D=1350&q%5Bprice%5D%5B%5D=2500&q%5Bregion_id%5D=&q%5Buser_id%5D='

for i in dic_comp.keys():
    print(len(dic_comp[i]))
dftapaz = pd.DataFrame.from_dict(dic_comp)

filename = "Users/hajiaga/Desktop/DataScience/tapazdata.csv"
filename2 = "Users/hajiaga/Desktop/DataScience/tapazdata.xlsx"
if os.path.exists(filename):
    os.remove(filename)
if os.path.exists(filename2):
    os.remove(filename2)
dftapaz.to_csv(filename, index=False) 
dftapaz.to_excel(filename2, index=False) 

dftapaz